In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
from matplotlib import rcParams

from sklearn.ensemble import RandomForestClassifier

import datetime
from operator import itemgetter
import csv

# Complete example using RandomForestClassifier

https://github.com/benhamner/ExpediaPersonalizedSortCompetition/

In [9]:
# load test set
test = pd.read_csv('../../training_set_VU_DM_2014.csv').fillna(value=0)

In [10]:
# obtain feature names and remove useless column date_time
feature_names = list(test.columns)
feature_names.remove("date_time")

# obtain feature values
features = test[feature_names].values

ids = test["srch_id"].unique()
np.random.shuffle(ids)
ids = ids[0:1000]

test = test.loc[test["srch_id"].isin(ids)]

test["rel"] = test["booking_bool"] + test["click_bool"]
test["rel"] = test["rel"].map({0:0,1:1,2:5})

# predict using trained model
print("Making predictions")
predictions = np.random.random(len(test))
predictions = list(-1.0*predictions)
recommendations = zip(test["srch_id"], test["prop_id"], test["booking_bool"], test["click_bool"], predictions,test["rel"])
    
rows = [(srch_id, prop_id,booking_bool,click_bool,rel)
    for srch_id, prop_id,booking_bool,click_bool, rank_float, rel
    in sorted(recommendations, key=itemgetter(0,4))]

with open("predict.csv", "w") as outfile:
    writer = csv.writer(outfile, lineterminator="\n")
    writer.writerow(("SearchId", "PropertyId", "BookingBool","ClickBool","rel"))
    writer.writerows(rows)

Making predictions


In [11]:
#https://www.kaggle.com/wendykan/ndcg-example
""" Reference from https://gist.github.com/bwhite/3726239
"""
def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum((2 ** r -1)/ np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

score = []
result = pd.read_csv('predict.csv')
score_n = 0
for srch_id in result["SearchId"].unique():
    #use NDCG@38 as per Kaggle site
    score_n += ndcg_at_k(result.loc[result["SearchId"]==srch_id,"rel"].values,38)
score.append(score_n / len(result["SearchId"].unique()))

In [12]:
print(score)

[0.35341455368709124]


In [ ]:
print(score)